In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

#### Fetch Mutual fund urls

In [2]:
url_list=[]
for i in range(0,78):
    r=requests.get('https://groww.in/slr/v1/search/derived/scheme?available_for_investment=true&doc_type=scheme&max_aum=&page={}&plan_type=Direct&q=&size=15&sort_by=0'.format(str(i)))
    data=r.json()
    df2=pd.DataFrame(data['content'])
    df2.head(1)
    df2['url']='https://groww.in/mutual-funds/'+df2.search_id
    url_list.extend(list(df2['url']))

In [3]:
url_list[0:20]

['https://groww.in/mutual-funds/quant-small-cap-fund-direct-plan-growth',
 'https://groww.in/mutual-funds/icici-prudential-commodities-fund-direct-growth',
 'https://groww.in/mutual-funds/quant-infrastructure-fund-direct-growth',
 'https://groww.in/mutual-funds/quant-tax-plan-direct-growth',
 'https://groww.in/mutual-funds/canara-robeco-small-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/boi-axa-small-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/quant-mid-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/quant-flexi-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/quant-active-fund-direct-growth',
 'https://groww.in/mutual-funds/pgim-india-midcap-opportunities-fund-direct-growth',
 'https://groww.in/mutual-funds/nippon-india-small-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/edelweiss-small-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/tata-small-cap-fund-direct-growth',
 'https://groww.in/mutual-funds/icici-prudential-techno

In [4]:
len(url_list)

1170

In [5]:
#### Fetch Mutual Data Investment information

In [6]:
from bs4 import BeautifulSoup
import re
import json


In [7]:
mutual_fund_data={}
for i in range(0,len(url_list)):
    r=requests.get(url_list[i])
    html_soup_object = BeautifulSoup(r.content, 'html.parser')
    matches=re.findall(r'window\.__INITIAL_STATE__.*<\/script>',r.text)
    filtered_data=matches[0].replace('window.__INITIAL_STATE__ = ','').replace('</script>','')
    holding_dict=json.loads(filtered_data)
    holding_dict['fundV2']['productPage']
    mutual_fund_name=list(holding_dict['fundV2']['productPage']['mfProductPageDataMap'].keys())[0]
    for hol_list in holding_dict['fundV2']['productPage']['mfProductPageDataMap'][mutual_fund_name]['MAIN_DATA']['holdings']:
        print(hol_list['company_name'],hol_list['sector_name'],hol_list['corpus_per'])
        if hol_list['company_name'] not in list(mutual_fund_data.keys()):
            mutual_fund_data[hol_list['company_name']]=[]
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])
        else:
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])
            
mutual_fund_data        

ITC Ltd. Consumer Staples 8.4597457
IRB Infrastructure Developers Ltd. Construction 6.12985469
RBL Bank Ltd. Financial 5.240721
Punjab National Bank Financial 4.60845693
HFCL Ltd. Communication 3.94874172
The India Cements Ltd. Construction 3.27376811
Hindustan Copper Ltd. Metals & Mining 2.99640797
TV18 Broadcast Ltd. Services 2.58409808
Archean Chemical Industries Ltd. Chemicals 2.21152511
Bikaji Foods International Ltd. Consumer Staples 2.18263997
Aegis Logistics Ltd. Services 2.13893051
E.I.D. - Parry (India) Ltd. Consumer Staples 2.1011624
Arvind Ltd. Textiles 2.07964873
Ambuja Cements Ltd. Construction 1.79994383
Just Dial Ltd. Services 1.79548684
Caplin Point Laboratories Ltd. Healthcare 1.74482836
Anand Rathi Wealth Ltd. Financial 1.72148926
Shree Renuka Sugars Ltd. Consumer Staples 1.65238273
Stylam Industries Ltd. Consumer Staples 1.59893767
Man Infraconstruction Ltd. Construction 1.51492931
Praj Industries Ltd. Capital Goods 1.49586504
Granules India Ltd. Healthcare 1.482637

IndexError: list index out of range

#### Run simple average operation on mutual fund data

In [8]:
import statistics

In [9]:
stats_mutual_data=[]
for key in mutual_fund_data.keys():
    #print(mutual_fund_data[key])
    average_data=[]
    for i in range(0,len(mutual_fund_data[key])):
        #print(mutual_fund_data[key][i][1])
        average_data.append(mutual_fund_data[key][i][1])
    #print(statistics.mean(average_data))    
        
    stats_mutual_data.append([len(mutual_fund_data[key]),key,statistics.mean(average_data)])
#stats_mutual_data 
stats_df=pd.DataFrame(stats_mutual_data,columns=['Number_of_mutual_fund','Company Name','Average_Contibution'])
stats_df.sort_values(['Number_of_mutual_fund','Average_Contibution'],ascending=[False, True]).head(60)

,Number_of_mutual_fund,Company Name,Average_Contibution
453,419,GOI,2.093068
191,325,ICICI Bank Ltd.,6.514452
96,318,HDFC Bank Ltd.,5.459283
92,294,State Bank of India,3.530612
259,273,Axis Bank Ltd.,3.275565
112,273,Infosys Ltd.,5.408471
89,262,Reliance Industries Ltd.,5.248419
105,250,Bharti Airtel Ltd.,2.967979
277,241,Larsen & Toubro Ltd.,3.378898
100,230,Housing Development Finance Corporation Ltd.,2.858552
